In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
import numpy as np
from scipy import stats
import joblib


def parse_input():
    dd = pd.ExcelFile('meta/COMBO PRO Banked Samples Aliquot Clinical Data.xlsx')
    kk = []
    for x in dd.sheet_names:
        tmp = pd.read_excel('meta/COMBO PRO Banked Samples Aliquot Clinical Data.xlsx', sheet_name=x)
        tmp['loc'] = x
        kk.append(tmp)
    tot = pd.concat(kk)
    return tot


def preprocess_metadata(flt):
    cols = "#E64B35B2", "#4DBBD5B2", "#00A087B2", "#3C5488B2", "#F39B7FB2", "#8491B4B2", "#91D1C2B2", "#DC0000B2", "#7E6148B2"
    cols = [x.lower() for x in list(cols)]

    meta = parse_input()
    dd = list(set(meta['COMBO Plasma Box Number']))
    meta = meta[meta['COMBO ID'].isin(flt)]
    enc = dict(zip(dd, cols))
    col_colors = meta['COMBO Plasma Box Number'].map(enc)

    col_colors.index = meta['COMBO ID']
    meta['COMBO Plasma Box Number'] = pd.Categorical(
        meta['COMBO Plasma Box Number'])

    meta['TB code'] = pd.Categorical(
        meta['TB Classification'])

    meta['code'] = meta['COMBO Plasma Box Number'].cat.codes
    meta['TB code'] = meta['TB code'].cat.codes
    meta.dropna(subset=['TB Classification'], inplace=True)
    return meta, col_colors


def get_prot_cl(std=True, nmr=False):
    X = pd.read_csv('data/protein_level_melted_merged.csv')
    X['Pr_gn'] = X['Protein.Group'] + '@' + X['Genes']
    X = pd.pivot_table(X, columns='COMBO ID', index='Pr_gn', values='value')
    meta, col_colors = preprocess_metadata(list(X))
    cls = meta[meta['TB Classification'].isin(['Confirmed TB', 'Unlikely TB'])]
    prot_cl = X[list(cls['COMBO ID'])].T
    if std:
        prot_cl = prot_cl.apply(lambda x: stats.zscore(x), axis=1)
    elif nmr: 
        prot_cl = pd.DataFrame(preprocessing.minmax_scale(prot_cl, feature_range=(0, 1), axis=1, copy=True), index=prot_cl.index, columns=prot_cl.columns)
    cls = dict(zip(cls['COMBO ID'], cls['TB Classification']))
    y = [cls[x] for x in list(prot_cl.index)]
    y = [1 if x == 'Confirmed TB' else 0 for x in y]
    return prot_cl, y


def get_unconfirmed(std=True, nmr=False):
    X = pd.read_csv('data/protein_level_melted_merged.csv')
    X['Pr_gn'] = X['Protein.Group'] + '@' + X['Genes']
    X = pd.pivot_table(X, columns='COMBO ID', index='Pr_gn', values='value')
    meta, col_colors = preprocess_metadata(list(X))

    cls = meta[meta['TB Classification'].isin(['Unconfirmed TB'])]
    prot_cl = X[list(cls['COMBO ID'])].T
    if std:
        prot_cl = prot_cl.apply(lambda x: stats.zscore(x), axis=1)
    elif nmr: 
        prot_cl = pd.DataFrame(preprocessing.minmax_scale(prot_cl, feature_range=(0, 1), axis=1, copy=True), index=prot_cl.index, columns=prot_cl.columns)
    cls = dict(zip(cls['COMBO ID'], cls['TB Classification']))
    y = [cls[x] for x in list(prot_cl.index)]
    y = [1 if x == 'Confirmed TB' else 0 for x in y]
    return prot_cl, y


def get_all(std=True, nmr=False):
    X = pd.read_csv('data/protein_level_melted_merged.csv')
    X['Pr_gn'] = X['Protein.Group'] + '@' + X['Genes']
    X = pd.pivot_table(X, columns='COMBO ID', index='Pr_gn', values='value')
    meta, col_colors = preprocess_metadata(list(X))
    prot_cl = X[list(meta['COMBO ID'])]
    if std:
        prot_cl = prot_cl.apply(lambda x: stats.zscore(x), axis=1)
    elif nmr: 
        prot_cl = pd.DataFrame(preprocessing.minmax_scale(prot_cl, feature_range=(0, 1), axis=1, copy=True), index=prot_cl.index, columns=prot_cl.columns)
    cls = dict(zip(meta['COMBO ID'], meta['TB Classification']))
    prot_cl = pd.melt(prot_cl, ignore_index=False)
    prot_cl['tb'] = prot_cl['COMBO ID'].map(cls)
    return prot_cl.reset_index()


t = 50
nanprot = pd.read_csv('new_biosignature/percentage_missing_allproteins_TBClassfiication.csv')
nanprot = nanprot[nanprot['TB Classification'].isin(['Confirmed TB', 'Unlikely TB'])]
nanprot = nanprot.groupby('Genes').mean(['% miss']).reset_index()
nanprot = nanprot[nanprot['% miss']<=t]

kk = pd.read_csv('new_biosignature/all_AUC_combined.csv')
ll = pd.read_csv('output/allAUC_lasso_6.csv')
ll['nfeat'] = 6
ll.columns = kk.columns
kk = pd.concat([ll, kk], ignore_index=True, axis=0)
kk['pr'] = [x.split(',') for x in kk['0']]
kk['gn'] = [[item.split('@')[1] for item in sublist] for sublist in kk['pr']]

kk['keep'] = [all(gene in nanprot['Genes'].tolist() for gene in x) for x in kk['gn']]
kk=kk[kk['keep']==True]
#kk2 = kk[(kk['nfeat'] == 6) & (kk['gn'].apply(set) == set(['WARS1', 'APOM', 'CD44', 'TNC', 'AMBP', 'MMP2']))]
#kk3 = kk[kk['nfeat'] != 6]
#kk = pd.concat([kk2,kk3])
kk = kk.loc[kk.groupby('nfeat')['1'].transform(max) == kk['1']]
kk = kk.loc[kk.groupby('nfeat')['2'].transform(max) == kk['2']]
kk


In [ ]:
## add the 1,2

for rr in [1,2]:
    ll = pd.read_csv('output/allAUC_lasso_{}.csv'.format(str(rr)))
    # Select the row with the maximum value in the 'prec70sesn' column
    ll = ll[ll['prec70sesn'] == ll['prec70sesn'].max()]
    ll = ll[ll['auc'] == ll['auc'].max()]
    ll.columns = list(kk)[:ll.shape[1]]
    ll['nfeat'] = rr
    ll['pr'] = [x.split(',') for x in ll['0']]
    ll['gn'] = [[item.split('@')[1] for item in sublist] for sublist in ll['pr']]
    ll['keep']=True
    kk = pd.concat([kk,ll])
    
kk.to_csv('new_biosignature/biosignature_output.csv')

In [ ]:

def test_auc_manual(nams, save=False):
    prot_cl, y = get_prot_cl(True, False)
    prot_cl = prot_cl[nams]
    X_train, X_test, y_train, y_test = train_test_split(
        prot_cl, y, random_state=1, stratify=y, test_size=0.25)
    clf = LogisticRegression(penalty='l2')
    if len(nams)==1:
        X_train = X_train.values.reshape(-1,1)
        X_test = X_test.values.reshape(-1,1)
    clf.fit(X_train, y_train)
    if save==True:
        joblib.dump(clf, "new_biosignature/model/model{}.pkl".format(str(len(nams))))
    y_pred_test = clf.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()
    fpr, tpr, thresholds = roc_curve(
        y_test, clf.predict_proba(X_test)[:, 1], pos_label=1)
    df = pd.DataFrame([fpr, tpr, thresholds]).T
    df.columns=['fpr', 'tpr', 'thresh']
    df['clf'] = '{} proteins (AUC = {})'.format(
        str(len(nams)), auc(fpr, tpr).round(3))
    return df


tt = []    
for _,i in kk.iterrows():
    tt.append(test_auc_manual(i['0'].split(','), save=True))
    
tt = pd.concat(tt)
tt.to_csv('new_biosignature/20250411_ROC_CV.csv')


### individual features AUC
xx = [x for x in kk['0']]
k = []
for x in xx:
    k.extend(x.split(','))
tt = []    
for i in set(k):
    tmp = test_auc_manual([i], save=False)
    tmp['clf'] = [x.replace('1 proteins', i.split('@')[-1]) for x in tmp['clf']]
    tt.append(tmp)
tt = pd.concat(tt)
tt.to_csv('new_biosignature/individual_AUC.csv')
    


In [ ]:
lassocoef = pd.read_csv('output/lassocoef.csv')

topn = []
for i in range(1,7):
    rocdf = test_auc_manual(lassocoef.head(i)['Pr_gn'], save=False)
    rocdf['t'] = 'TopN'
    rocdf['n'] = i
    topn.append(rocdf)

topn = pd.concat(topn)
tt['n'] = [x.split(' ')[0] for x in tt['clf']]
tt['t'] = 'BestN'
tot = pd.concat([topn, tt])
tot.to_csv('new_biosignature/topN_vs_bestN.csv')

In [ ]:
from statsmodels.stats.proportion import proportion_confint


prot_cl, y = get_prot_cl(True, False)
thresh = {}
for i in range(3,7):
    clf = joblib.load('new_biosignature/model/model{}.pkl'.format(str(i)))
    print(clf.feature_names_in_)
    X_train, X_test, y_train, y_true = train_test_split(
    prot_cl[clf.feature_names_in_], y, random_state=1, stratify=y, test_size=0.25)

    y_pred_proba = clf.predict_proba(X_test)[:,1]
   # Step 1: Find the threshold for 70% specificity
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    specificities = 1 - fpr
    # Find the threshold that gives approximately 70% specificity
    target_specificity = 0.7
    closest_idx = np.argmin(np.abs(specificities - target_specificity))
    threshold_for_70_specificity = thresholds[closest_idx]
    y_pred_proba = (y_pred_proba >= threshold_for_70_specificity).astype(int)
    thresh[i] = threshold_for_70_specificity
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred_proba >= threshold_for_70_specificity).ravel()
    
    sensitivity = tp / (tp + fn)
    sensitivity_ci = proportion_confint(count=tp, nobs=tp + fn, alpha=0.05, method='beta')

    # Specificity calculation and 95% CI
    specificity = tn / (tn + fp)
    specificity_ci = proportion_confint(count=tn, nobs=tn + fp, alpha=0.05, method='beta')

    print(sensitivity_ci, specificity_ci)

thresh

In [ ]:
## plot biosignature across all TB classes
import pandas as pd
import numpy as np


def norm_to_unlikely(grp):
    sub = grp[grp['tb']=='Unlikely TB']
    grp['value_norm'] = grp['value'] - np.mean(sub['value'].values)
    return grp


rocv = pd.read_csv('new_biosignature/biosignature_output.csv')
rocv = rocv[rocv['nfeat'].isin([5,6])]
rocv = [x.split(',') for x in rocv['0']]
i = set([x for xs in rocv for x in xs])
dd = get_all()
i = set([x for xs in rocv for x in xs])
dd2 = dd[dd['Pr_gn'].isin(i)]
dd2 = dd2.groupby(['Pr_gn']).apply(norm_to_unlikely).reset_index(drop=True)
dd2['Pr_gn'] = [x.split('@')[-1] for x in dd2['Pr_gn']]
dd2= dd2.groupby(['Pr_gn', 'tb'])['value_norm'].describe()
dd2.to_csv('new_biosignature/biosignature_clinical_sites.csv')
dd2

In [ ]:
import glob
import pandas as pd
import numpy as np
import joblib



prot_cl = get_all()
dd = pd.read_csv('../meta/combo_ms_names_test.csv')
dd = dd[dd['TB Classification']=='Unconfirmed TB']
prot_cl = prot_cl[prot_cl['COMBO ID'].isin(dd['COMBO ID'])]

tt = []
preds = []
idss = pd.read_csv('new_biosignature/biosignature_output.csv')
idss = [x.split(',') for x in list(set(idss['0']))]
idss = dict(zip([len(x) for x  in idss], idss))
prot_cl = pd.pivot_table(data=prot_cl, columns='Pr_gn', index='COMBO ID', values='value')
for i in range(3,7):
    clf = joblib.load('new_biosignature/model/model{}.pkl'.format(str(i)))
    print(clf.feature_names_in_)
    tmp = prot_cl[idss[i]]
    ypred = pd.DataFrame(clf.predict_proba(tmp))
    ## utilize prediction threshold for reclassifying positive and neg TB
    #ypred[0] = [0 if x<=thresh[i] else 1 for x in ypred[1]]
    ypred[0] = [0 if x<=0.5 else 1 for x in ypred[1]]
    #ypred = pd.DataFrame(clf.predict(tmp))
    # print(ypred)
    # assert False
    ypred['sid'] = prot_cl.index
    ypred['cc'] = i
    ytot = ypred.groupby([0]).size().to_frame()
    ytot['cc'] = '{} feature LR model'.format(i)
    tt.append(ytot)
    preds.append(ypred)    

## re-classify all unconfirmed group

tt = pd.concat(tt)
ff = pd.concat(preds)
ff.columns=['Class', 'LR probability', 'COMBO ID', 'model feature number']
ff.to_csv('prediction/pred_model.csv')
tt.columns = ['nr', 'cc']
## add GS data
preds=pd.concat(preds)

tt.reset_index(inplace=True)
tt[0].replace({0:'Negative', 1:'Positive'}, inplace=True)

# #tt[0]=tt[0].
#tt

preds.to_csv('new_biosignature/individual_prediction.csv')
tt.to_csv('new_biosignature/pred_count.csv')


In [ ]:
from upsetplot import from_contents, plot
from upsetplot import UpSet
import matplotlib.pyplot as plt



preds[0]= preds[0].replace({0:'Negative', 1:'Positive'})
pr_tmp = preds[preds[0]=='Positive']
kk = {}
for i,grp in pr_tmp.groupby('cc'):
    kk[str(i)]=list(grp['sid'])

toplot = from_contents(kk)


fig = plt.figure(figsize=(2.5, 4))
upset = UpSet(toplot, sort_by='cardinality', show_counts=True)  # disable the default bar chart

upset.plot(fig=fig)

plt.savefig('revision_figures/Fig5_B.pdf', bbox_inches='tight', dpi=800)
plt.savefig('revision_figures/Fig5_B.svg', bbox_inches='tight', dpi=800)
plt.close()

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis

def theme_Publication():
    sns.set_style("ticks")
    sns.set_context("paper", font_scale=1.2)
    sns.set_palette("colorblind")
    
    plt.rcParams["font.family"] = "Helvetica"
    plt.rcParams["figure.figsize"] = [1.5, 1.5]
    plt.rcParams["axes.linewidth"] = 1
    plt.rcParams["axes.edgecolor"] = "black"
    plt.rcParams["legend.frameon"] = False
    plt.rcParams["legend.loc"] = "lower center"
    plt.rcParams["legend.framealpha"] = 1
    plt.rcParams["legend.fontsize"] = 9
    plt.rcParams["legend.title_fontsize"] = 9
    plt.rcParams["axes.grid"] = False
    plt.rcParams["figure.subplot.left"] = 0
    plt.rcParams["figure.subplot.right"] = 1
    plt.rcParams["figure.subplot.bottom"] = 0
    plt.rcParams["figure.subplot.top"] = 1

    return None


tot2 = get_all(False, False)
dd = pd.read_csv('../meta/combo_ms_names_test.csv')
dd = dd[dd['TB Classification'].isin(['Unconfirmed TB', 'Confirmed TB'])]
tot2 = tot2[tot2['COMBO ID'].isin(dd['COMBO ID'])]
tmp = tot2[tot2['tb']=='Confirmed TB']
z = dict(zip(tmp['COMBO ID'], tmp['tb']))
tot2 = pd.pivot_table(data=tot2, columns='Pr_gn', index='COMBO ID', values='value')
tt = {}
nrmodels = len(set(preds[0]))
for i, grp in preds.groupby(['sid']):
    tt[i[0]] = str(grp[grp[0]=='Positive'].shape[0])
tt.update(z)
tot3 = tot2
Xarr = StandardScaler(with_mean=True).fit_transform(tot3)

clf = LinearDiscriminantAnalysis(solver='svd', n_components=2).fit(Xarr, [tt[i] for i in tot2.index])
#print(clf.explained_variance_ratio_)
lda_df = clf.transform(Xarr)


X_reduced = pd.DataFrame(lda_df)
X_reduced['variable'] = tot3.index
X_reduced['class'] = X_reduced['variable'].map(tt)
X_reduced['class'] = X_reduced['class'].replace({'0':'Negative Unconfirmed', '1': 'Positive 1/4 models', '2': 'Positive 2/4 models', '3': 'Positive 3/4 models', '4': 'Positive all models'})
X_reduced['nrfreat'] = str(i)
X_reduced.to_csv('new_biosignature/lda_projection.csv')


# ## LDA loadings per feature for supplementary
lda_loadings = pd.DataFrame(clf.scalings_, index=list(tot3)).reset_index()
totfeat = []
for i in range(3,7):
    clf = joblib.load('new_biosignature/model/model{}.pkl'.format(str(i)))
    print(clf.feature_names_in_)
    totfeat.extend(clf.feature_names_in_)
totfeat = set(totfeat)
lda_loadings['biosignature'] = lda_loadings['index'].isin(totfeat)
lda_loadings[0] = np.abs(lda_loadings[0])
lda_loadings['rank'] = lda_loadings[0].rank(ascending=False)
lda_loadings[['Protein', 'Gene']] = lda_loadings['index'].str.split('@', expand=True)
lda_loadings.drop(['index'], axis=1, inplace=True)
lda_loadings.to_csv('new_biosignature/lda_loadings.csv')


In [ ]:
# biosignature for health and latent TB pca
tot2 = get_all(False, False)
dd = pd.read_csv('../meta/combo_ms_names_test.csv')
dd = dd[dd['TB Classification'].isin(['Latent TB', 'Healthy', ])]
tot2 = tot2[tot2['COMBO ID'].isin(dd['COMBO ID'])]
tt = dict(zip(tot2['COMBO ID'], tot2['tb']))
tot2 = pd.pivot_table(data=tot2, columns='Pr_gn', index='COMBO ID', values='value')

test = []
for i in range(3,7):
    clf = joblib.load('new_biosignature/model/model{}.pkl'.format(str(i)))
    tot3 = tot2[clf.feature_names_in_]
    Xarr = StandardScaler(with_mean=True).fit_transform(tot3)
    pca_df = PCA(n_components=2, svd_solver='full').fit_transform(Xarr)
    X_reduced = pd.DataFrame(pca_df)
    X_reduced['variable'] = tot2.index
    X_reduced['class'] = X_reduced['variable'].map(tt)
    X_reduced['nrfreat'] = str(i)
    
    test.append(X_reduced)
   
test = pd.concat(test)
test.to_csv('new_biosignature/healthy_latent_pca.csv')
    

,0,1,variable,class,nrfreat
0,0.458517,0.295461,C138,Healthy,3
1,0.521191,1.437190,C139,Healthy,3
2,-0.299973,0.594768,C140,Healthy,3
3,-0.715022,-0.306298,C141,Healthy,3
4,-0.130670,-1.505676,C142,Healthy,3
...,...,...,...,...,...
22,-2.768197,-1.088194,C160,Healthy,6
23,-1.981232,0.842064,C161,Healthy,6
24,0.569850,1.294248,C162,Latent TB,6
25,-2.156477,0.416271,C163,Latent TB,6
